In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from wpgnn import WPGNN
from graph_nets.utils_tf import *
from graph_nets.utils_np import graphs_tuple_to_data_dicts
from playgen import PLayGen
import utils

In [ ]:
# Load data
scale_factors = {'x_globals': np.array([[0., 25.], [0., 25.], [0.09, 0.03]]),
                   'x_nodes': np.array([[0., 75000.], [0., 85000.], [15., 15.]]),
                   'x_edges': np.array([[-100000., 100000.], [0., 75000.]]),
                 'f_globals': np.array([[0., 500000000.], [0., 100000.]]),
                   'f_nodes': np.array([[0., 5000000.], [0.,25.]]),
                   'f_edges': np.array([[0., 0.]])}
data = utils.load_data('example_data.h5', normalize=True, scale_factors=scale_factors)

# Initialize WPGNN model
model = WPGNN(model_path='model/wpgnn.h5')

In [ ]:
# Run 10 epochs of training
model.fit(data, data, learning_rate=1e-4, decay_rate=0.999, batch_size=25,
          epochs=10, print_every=1, save_every=5,
          save_model_path='example_training')

In [ ]:
# Initialize wind plant layout generator
generator = PLayGen()

# Generate random wind farm layout
xy = generator()
yaw_angles = np.zeros((xy.shape[0], 1))

# Build graph input for model
wind_speed, wind_direction, turb_intensity = 8., 45., 0.08
uv = utils.speed_to_velocity([wind_speed, wind_direction])

edges, senders, receivers = utils.identify_edges(xy, wind_direction)
input_graph = [{'globals': np.array([uv[0], uv[1], turb_intensity]),
                  'nodes': np.concatenate((xy, yaw_angles), axis=1),
                  'edges': edges,
                'senders': senders,
              'receivers': receivers}]

# Evaluate model
normed_input_graph, _ = utils.norm_data(xx=input_graph, scale_factors=scale_factors)
normed_output_graph = graphs_tuple_to_data_dicts(model(data_dicts_to_graphs_tuple(normed_input_graph)))
output_graph = utils.unnorm_data(ff=normed_output_graph, scale_factors=scale_factors)

plt.figure()
plt.scatter(input_graph[0]['nodes'][:, 0], input_graph[0]['nodes'][:, 1],
            c=output_graph[0]['nodes'][:, 0]/1e6, edgecolor='k')
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()
plt.xlim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.ylim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.gca().set_aspect(1.)
cbar = plt.colorbar()
cbar.set_label('Power (MW)')
plt.show()

In [ ]:
# Generate random wind farm layout
xy = generator()
yaw_angles = np.zeros((xy.shape[0], 1))

# Build graph input for model
wind_speed, wind_direction = np.linspace(0, 20, 21), np.linspace(0, 360, 73)
turb_intensity = 0.08

# Create list of graph data
input_graph = []
for i, wd_i in enumerate(wind_direction):
    for j, ws_j in enumerate(wind_speed):
        uv = utils.speed_to_velocity([ws_j, wd_i])
        edges, senders, receivers = utils.identify_edges(xy, wd_i)
        input_graph.append({'globals': np.array([uv[0], uv[1], turb_intensity]),
                              'nodes': np.concatenate((xy, yaw_angles), axis=1),
                              'edges': edges,
                            'senders': senders,
                          'receivers': receivers})
# Evaluate model
normed_input_graph, _ = utils.norm_data(xx=input_graph, scale_factors=scale_factors)
normed_output_graph = graphs_tuple_to_data_dicts(model(data_dicts_to_graphs_tuple(normed_input_graph)))
output_graph = utils.unnorm_data(ff=normed_output_graph, scale_factors=scale_factors)

# Extract power data
P = np.zeros((wind_direction.size, wind_speed.size))
for i, wd_i in enumerate(wind_direction):
    for j, ws_j in enumerate(wind_speed):
        P[i, j] = output_graph[i*wind_speed.size + j]['globals'][0]

        
plt.figure(figsize=(12, 3))

plt.subplot(121)
plt.scatter(input_graph[0]['nodes'][:, 0], input_graph[0]['nodes'][:, 1], facecolor='b', edgecolor='k')
xlim = plt.gca().get_xlim()
ylim = plt.gca().get_ylim()
plt.xlim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.ylim(np.minimum(xlim[0], ylim[0]), np.maximum(xlim[1], ylim[1]))
plt.gca().set_aspect(1.)

plt.subplot(122)
plt.contourf(wind_direction, wind_speed, P.T/1e6, levels=15)
plt.gca().set_aspect(10.)
plt.xlabel('Wind Direction')
plt.ylabel('Wind Speed')
cbar = plt.colorbar()
cbar.set_label('Power (MW)')

plt.show()